In [7]:
import pandas as pd

data = pd.read_csv('Ontario_CPI.csv')
# reformat date values
data['Date'] = '20' + data['Date']
data.head()

,Date,All-items,Food,Shelter,"Household operations, furnishings and equipment",Clothing and footwear,Transportation,Gasoline,Health and personal care,"Recreation, education and reading","Alcoholic beverages, tobacco products and recreational cannabis",All-items excluding food and energy,All-items excluding energy,Energy,Goods,Services
0,2001-Jan,96.5,95.0,97.3,96.9,102.4,98.2,103.8,97.7,95.6,80.3,96.1,95.9,102.3,97.9,95.3
1,2001-Feb,97.1,95.9,97.4,97.5,103.2,98.6,103.4,98.3,97.1,80.6,96.7,96.6,101.9,98.3,96.0
2,2001-Mar,97.7,96.6,98.9,97.8,105.3,97.4,100.6,98.4,98.2,80.5,97.0,97.0,104.8,99.4,96.2
3,2001-Apr,98.1,96.8,98.8,98.7,101.0,99.3,110.1,98.7,98.8,84.7,97.2,97.1,108.4,100.1,96.5
4,2001-May,98.7,97.7,99.0,98.8,100.4,100.4,115.3,98.9,99.6,86.0,97.4,97.5,110.7,100.8,96.8


In [12]:
import altair as alt

# plot visualization to analyze task 1 - overall CPI trends across timeline in Ontario
time_series = alt.Chart(data).mark_point().encode(
    x='Date:T',
    y=alt.Y('All-items:Q', axis=alt.Axis(title='Consumer Price Index'))
).properties(
    width=800,
    height=400,
    title='Consumer Price Index in Ontario (2001-2024)'
).interactive()

# save and export time series graph in PNG format
time_series.save('visualization_1.html')

# visualize
time_series

# yearly_data = data
# yearly_data['Date'] = pd.to_datetime(yearly_data['Date'], format='%Y-%b')
# yearly_data['Year'] = data['Date'].dt.year
# yearly_data['CPI_change'] = data.groupby('Year')['All-items'].diff()
# yearly_data
# # add a second line representing net change in CPI year-to-year
# time_series_change = alt.Chart(yearly_data).mark_point().encode(
#     x='Year:O',
#     y=alt.Y('CPI_change:Q', axis=alt.Axis(title='Consumer Price Index Change'))
# )

# # layer the two lines together
# time_series_res = alt.layer(time_series, time_series_change)
# time_series_res

alt.Chart(...)

In [13]:
# setup data for visualization to analyze task 2 - first compute category means
filtered_data = data.drop(columns=['All-items excluding food and energy', 'All-items excluding energy'])
cpi_avgs = filtered_data.mean()
# format data prophttp://localhost:8888/notebooks/Data_Visualization_Fundamentals_Final.ipynb#erly to plot average CPI data
cpi_avgs_data = pd.DataFrame(cpi_avgs, columns=['CPI_avg']).reset_index()
cpi_avgs_data.rename(columns={'index': 'Product Sector'}, inplace=True)

# add interactive visualization elements with selection to be used as a filter
selection = alt.selection_point(encodings=['color'])

# plot an overview visualization of average CPIs before adding linked interactive chart
overview_chart = alt.Chart(cpi_avgs_data).mark_bar().encode(
    x=alt.X('CPI_avg:Q', axis=alt.Axis(title='Average CPI Across 2001-2024')),
    y=alt.Y('Product Sector:N', axis=alt.Axis(title='Product Sectors')),
    tooltip=['Product Sector:N', 'CPI_avg:Q'],
    color=alt.condition(selection, alt.Color('Product Sector:N'), alt.value('lightgray')) # disable color if selection is made
).properties(
    title='Average Consumer Price Index in Ontario Across Sectors'
).add_params(
    selection
)
# overview_chart

# build line-chart showing time series price index for selected categories
# first requires a pivot of data formatting to plot CPI data across all categories
pivoted_data = filtered_data.melt(id_vars='Date', var_name='Product Sector', value_name='CPI')[:-1]
selected_chart = alt.Chart(pivoted_data).mark_line().encode(
    x='Date:T',
    y=alt.Y('CPI:Q', axis=alt.Axis(title='Consumer Price Index')),
    color='Product Sector:N'
).properties(
    width=500,
    height=400,
    title='Consumer Price Index in Ontario Across Sectors'
).transform_filter(
    selection
).interactive()

# # combine detailed chart with overview chart together for interactivity
combined_chart = overview_chart & selected_chart

# save and export linked graph in HTML format
combined_chart.save('visualization_2.html')

# visualize
combined_chart

alt.VConcatChart(...)

In [14]:
# final visualization (heatmap) to address the 3rd task of exploring any monthly trends in spending
# convert date to datetime format
filtered_data['Date'] = pd.to_datetime(filtered_data['Date'], format='%Y-%b')
filtered_data2 = filtered_data.drop(columns=['Goods', 'Services', 'Clothing and footwear'])
# pivot data formatting as before
formatted_data = filtered_data2.melt(id_vars='Date', var_name='Product Sector', value_name='CPI')
# formatted_data

# add brush tool selection
brush = alt.selection_interval()

# create a heatmap visualization in order to group monthly CPI data for sectors
monthly_heatmap = alt.Chart(formatted_data).mark_rect().encode(
    x=alt.X('month(Date):O', axis=alt.Axis(title='Month')),
    y=alt.Y('Product Sector:N', axis=alt.Axis(title='Product Sector')),
    color=alt.condition(brush, alt.Color('CPI:Q', scale=alt.Scale(scheme='inferno'), sort='descending'), alt.value('lightgray')),
    tooltip=['month(Date):O', 'Product Sector:N', 'CPI:Q']
).properties(
    width=700,
    height=400,
    title='Monthly Consumer Price Index in Ontario'
).add_params(
    brush
)

# save and export graph in HTML format
monthly_heatmap.save('visualization_3.html')

# visualize
monthly_heatmap

alt.Chart(...)